# OCR in Python mit PyTesseract 


In diesem Notizbuch werden wir pyTesseract ausführen, um maschinenlesbaren Text zu erzeugen aus:
* einem JPEG-Bild
* einem mehrseitigen PDF
* einem Korpus mehrseitiger PDFs

<!--In this notebook, we will run pyTesseract to produce machine readable text from:
* a JPEG image
* a multi-paged PDF
* a corpus of multi-page PDF-s-->

## Importieren von Tools <!--Importing tools-->

In [ ]:
import pytesseract
from PIL import Image

## Verarbeitung eines Bildes <!--Processing one image-->

<img src='grippe.jpeg' width="600">

So können wir **OCR auf dieses Bild** des Zeitungsartikels ('Die Grippe wütet weiter') durchführen: <!-- This is how we can **perform OCR on this image** of the ('*Die Grippe wütet weiter*') newspaper article: -->

In [ ]:
ocr_output = pytesseract.image_to_string(Image.open('grippe.jpeg'), lang='frk') 
print(ocr_output)

## Verarbeitung eines (mehrseitigen) PDFs

Mit ein wenig mehr Python-Code können wir pytesseract auch verwenden, um gesamte **PDF-Dateien mit vielen Seiten** zu OCRen: <!-- With a bit more Python code, we can also use pytesseract to OCR entire **PDF files with many pages**: -->

In [ ]:
from pathlib import Path
from pdf2image import convert_from_path
from tqdm import tqdm

In [ ]:
sample_pdf_path = Path('../data/pdf/SNP27112366-19181224-0-0-0-0.pdf')
recognized_pages = []
converted_pdf = tqdm(convert_from_path(sample_pdf_path, use_cropbox=True))
for image in converted_pdf:
    recognized = pytesseract.image_to_string(image, 
                                             lang='frk') 
    #print(recognized)
    recognized_pages.append(recognized)

Schauen wir uns die erste Seite an: <!-- Let's look at the first page:-->

In [ ]:
print(recognized_pages[0])

Letzte Seite: <!-- Last page: --> 

In [ ]:
print(recognized_pages[-1])

Keines dieser Ergebnisse sieht besonders gut aus (hauptsächlich aufgrund der Scan-Qualität und allgemeiner Herausforderungen bei der Arbeit mit alten Zeitungen). In den nächsten Abschnitten werden wir lernen, wie man
* a) die OCR-Qualität misst
* b) die Qualität in der OCR-Nachkorrekturphase verbessert

<!-- None of these results look very good (mostly due to scan quality and general challenges of working with old newspapers). In the next parts we will learn how to 
* a) measure the OCR quality
* b) improve the quality at the OCR postcorrection stage -->

Um die OCR-Funktion auf einer anderen PDF-Datei auszuführen, müssen Sie in der obigen Zeile einen Dateipfad dazu angeben: `sample_pdf_path = Path('../data/pdf/SNP27112366-19181224-0-0-0-0.pdf')`. 

<details>
  <summary><b>Was ist ein Dateipfad?</b> (klicken)</summary>

Ein **Dateipfad** ist eine Zeichenkette, die deinem Programm sagt, wo eine Datei auf deinem Computer oder Server gespeichert ist. Er hilft dem Programm, Dateien zu finden und auf sie zuzugreifen, um sie zu lesen, zu schreiben oder zu bearbeiten.

#### Arten von Dateipfaden:
1. **Absoluter Dateipfad**:  
   Ein absoluter Pfad gibt den vollständigen Speicherort einer Datei ausgehend vom Stammverzeichnis deines Systems an.
   - Beispiel unter Windows:  
     `C:\Users\JohnDoe\Documents\file.txt`
   - Beispiel unter macOS/Linux:  
     `/Users/JohnDoe/Documents/file.txt`
     
2. **Relativer Dateipfad**:  
   Ein relativer Pfad zeigt dem Programm, wie es eine Datei basierend auf dem aktuellen Arbeitsverzeichnis (dem Ordner, in dem dein Skript ausgeführt wird) finden kann.
   - Beispiel:  
     `Documents/file.txt`  
     (Dies sucht die Datei in einem Ordner namens `Documents` innerhalb des aktuellen Verzeichnisses).

#### Pfadtrennzeichen:
- Unter Windows verwenden Pfade Backslashes (`\`):  
  `C:\folder\file.txt`
- Unter macOS/Linux verwenden Pfade Schrägstriche (`/`):  
  `/folder/file.txt`

#### Beispiel in Python:

```
# Absoluter Pfad
file = open('C:/Users/JohnDoe/Documents/file.txt')
    
# Relativer Pfad
file = open('Documents/file.txt')
```

Python bietet auch Tools, um Pfade so zu handhaben, dass sie auf jedem Betriebssystem funktionieren, wie die Module `os` und `pathlib`. Wir verwenden oben `pathlib`, damit dieses Notebook auf jedem Rechner funktioniert. Dadurch können wir Pfade im Unix-Stil schreiben.
    
</details>

## (Advanced) Verarbeitung des gesamten Korpus von PDFs mit derselben OCR-Engine <!-- (Advanced) Processing the whole corpus of PDF-s with the same OCR engine -->

Der untenstehende Code verarbeitet alle Dateien im Ordner `'../data/pdf'`, die die Endung '.pdf' haben, und speichert die Ergebnisse dann im Ordner `'../data/txt'` (die Dateinamen bleiben gleich, aber mit der Endung '.txt' anstelle von '.pdf'). **WARNUNG**: Bei einer großen Anzahl (>5) von PDFs wird dies viel Zeit in Anspruch nehmen.

<!-- The code below will process all the files in folder `'../data/pdf'` which have '.pdf' as extension, and then put the results into the `'../data/txt'` (the filenames will be the same but with '.txt' extension instead of '.pdf'). **WARNING**: For a large (>5) number of PDF-s this will take a long time. -->

In [ ]:
pathpdf = Path('../data/pdf')
pathtxt = Path('../data/txt')

In [ ]:
for filename in tqdm(pathpdf.iterdir()):
    if filename.suffix == '.pdf':
        converted_pdf = convert_from_path(filename, use_cropbox=True)
        output_path = pathtxt / filename.stem 
        output_path = output_path.with_suffix('.txt')
        with output_path.open('w') as output_txt:
            for image in converted_pdf:
                recognized = pytesseract.image_to_string(image, 
                                                         lang='frk') 
                output_txt.write(recognized)